In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATv2Conv

from GraphDataset import MyDataset
from torch_geometric.loader import DataLoader


In [2]:
# Load soft labels with membership level to each cover

df_data = pd.read_csv('data/ReyZamuro_softLabels.csv',index_col=0)
df_data = df_data.drop('RZUB02')
df_data = df_data.drop('RZUA03b')
print(df_data.head())

                   1         2         3
field_numb                              
RZUA01      1.000000  0.000000  0.000000
RZUA02      1.000000  0.000000  0.000000
RZUA04      0.589278  0.410722  0.000000
RZUA05      0.892664  0.000000  0.107336
RZUA06      0.938423  0.000000  0.061577


In [3]:
#Create list of ARUs and labels

DatosN = list(df_data.index)
Clases = df_data.values.argmax(1)
etiquetasN = Clases

In [4]:
features = 'YAMNet'#'PANNs'#'YAMNet' #'VGGish'#'AI'

train_dataset = MyDataset(ListaArchivos=DatosN,
                          etiquetas=etiquetasN, caract=features)
print(len(train_dataset))

92


In [5]:
unpacked_data = [train_dataset[i][0] for i in range(len(train_dataset))]
x = torch.stack(unpacked_data, dim=0).transpose(0,1)

0
RZUA01
file_path='/media/andrescastro/Seagate Backup Plus Drive/AECO/DeepFeatures_data/ReyZamuro/RZUA01_yamnet.pickle'
n_feat=1024
10
11
12
13
14
15
16
17
18
19
20
21
22
23
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
0
1
2
3
4
5
6
7
8
9
Data1.shape=torch.Size([3, 24, 1024])
1
RZUA02
file_path='/media/andrescastro/Seagate Backup Plus Drive/AECO/DeepFeatures_data/ReyZamuro/RZUA02_yamnet.pickle'
n_feat=1024
10
11
12
13
14
15
16
17
18
19
20
21
22
23
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
0
1
2
3
4
5
6
7
8
9
Data1.shape=torch.Size([3, 24, 1024])
2
RZUA04
file_path='/media/andrescastro/Seagate Backup Plus Drive/AECO/DeepFeatures_data/ReyZamuro/RZUA04_yamnet.pickle'
n_feat=1024
10
11
12
13
14
15
16
17
18
19
20
21
22
23
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19

In [6]:
x.shape

torch.Size([3, 92, 24, 1024])

## Edge Creation

In [7]:
from utils import edge_creation_nodeinfo, is_connected, edge_creation_geoDistance, plot_distance_matrix_heatmap, edge_creation_coverinfo
from torch_geometric.utils import is_undirected

In [8]:
graphs = edge_creation_coverinfo(torch.tensor(df_data.values), x,'knn', k_neigh=11)
for i in graphs:
    print(f"Is the graph {i} connected? {is_connected(i)}")
    print(f'Is the graph undirected {is_undirected(i.edge_index)}')

Is the graph Data(x=[92, 24, 1024], edge_index=[2, 1260]) connected? True
Is the graph undirected True
Is the graph Data(x=[92, 24, 1024], edge_index=[2, 1260]) connected? True
Is the graph undirected True
Is the graph Data(x=[92, 24, 1024], edge_index=[2, 1260]) connected? True
Is the graph undirected True


In [9]:
train_loader = DataLoader(graphs, batch_size=1, shuffle=False)

## Crear modelo y entrenar

In [10]:
class MatrixGCNVAE(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, latent_dim, num_nodes, heads=1):
        super(MatrixGCNVAE, self).__init__()
        
        # Encoder components
        self.conv1d = torch.nn.Conv1d(1, 64, 24, stride=24)
        self.gat1 = GATv2Conv(in_channels, hidden_channels, heads=heads)
        
        # Mean and log variance layers for the latent distribution
        self.bn = torch.nn.BatchNorm1d(hidden_channels)
        self.fc_mu = GATv2Conv(hidden_channels*heads, latent_dim, heads=1)
        self.fc_logvar = GATv2Conv(hidden_channels*heads, latent_dim, heads=1)
        
        # Decoder components for node features
        self.decoder_fc1 = torch.nn.Linear(latent_dim, hidden_channels)
        self.num_rec = int(in_channels*24/64)
        self.decoder_fc2 = torch.nn.Linear(hidden_channels, self.num_rec)
        
        # Decoder components for adjacency matrix
        self.num_nodes = num_nodes
        self.adj_decoder = torch.nn.Sequential(
            torch.nn.Linear(latent_dim, hidden_channels),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden_channels, num_nodes)
        )
        
    def encode(self, x, edge_index):
        # Process input features
        x = x.transpose(1, 2).flatten(1)  # Flatten the matrix features
        x = self.conv1d(x.unsqueeze(1))
        x = x.view(x.size(0), -1)
        
        # GCN encoding
        x = self.gat1(x, edge_index)
        x = F.relu(x)
        
        # Get latent distribution parameters
        x = self.bn(x)
        mu = self.fc_mu(x, edge_index)
        logvar = self.fc_logvar(x, edge_index)
        
        return mu, logvar
    
    def reparameterize(self, mu, logvar):
        return mu + torch.randn_like(logvar) * torch.exp(logvar)
    
    def decode(self, z):
        # Decode node features from latent space
        h = F.relu(self.decoder_fc1(z))
        node_reconstruction = self.decoder_fc2(h)
        
        # Decode adjacency matrix from latent space
        adj_logits = self.adj_decoder(z)
        # Create adjacency predictions - a matrix where each row i contains scores for edges from node i to all nodes
        adj_matrix = torch.sigmoid(torch.matmul(adj_logits, adj_logits.transpose(0, 1)))
        
        return node_reconstruction, adj_matrix
    
    def forward(self, x, edge_index):
        # Encode to get latent distribution
        mu, logvar = self.encode(x, edge_index)
        # mu, logvar = torch.tanh(mu), torch.tanh(logvar)
        # Sample from the latent distribution
        z = self.reparameterize(mu, logvar)
        # print(z)
        # Decode to get reconstructions
        node_reconstruction, adj_reconstruction = self.decode(z)
        
        return node_reconstruction, adj_reconstruction, mu, logvar
    
    def edge_index_to_adj_matrix(self, edge_index, num_nodes):
        """Convert edge_index to dense adjacency matrix"""
        adj_matrix = torch.zeros((num_nodes, num_nodes), device=edge_index.device)
        adj_matrix[edge_index[0], edge_index[1]] = 1.0
        return adj_matrix
    
    def loss_function(self, node_reconstruction, adj_reconstruction, x_original, edge_index, mu, logvar, alpha=1.0, beta=1.0):
        # Node feature reconstruction loss (MSE)
        # print(f"true reconstructed= {node_reconstruction.shape}, original={x_original.transpose(1, 2).flatten(1).shape}")
        feature_loss = F.mse_loss(node_reconstruction, x_original.transpose(1, 2).flatten(1))
        
        # Adjacency matrix reconstruction loss (BCE)
        true_adj = self.edge_index_to_adj_matrix(edge_index, self.num_nodes)
        
        # Binary cross entropy for adjacency matrix
        # We can add class weights if the graph is sparse
        # print(f"{adj_reconstruction.double()=}")
        # print(f"{true_adj.double()=}")
        adj_loss = F.binary_cross_entropy(adj_reconstruction.double(), true_adj.double())
        
        # KL Divergence loss
        kl_loss = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
        
        # Total loss with weighting factors
        total_loss = feature_loss + alpha * adj_loss + beta * kl_loss
        
        return total_loss, feature_loss, adj_loss, kl_loss

In [11]:
max_nodes = max([data.num_nodes for data in graphs])
num_feat = graphs[0].x.shape[-1]
num_feat

1024

## Modelo

In [ ]:
model = MatrixGCNVAE(in_channels=64*num_feat, hidden_channels=4*num_feat, latent_dim=int(num_feat/4), num_nodes=max_nodes)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01) #0.01 GCN

# Training function
def train():
    model.train()
    model.double()
    for graph in train_loader:
        optimizer.zero_grad()
        node_reconstruction, adj_reconstruction, mu, logvar = model(graph.x.double(), graph.edge_index)
        loss, feature_loss, adj_loss, kl_loss = model.loss_function(node_reconstruction, adj_reconstruction, graph.x.double(),graph.edge_index, mu, logvar, alpha=1.0, beta=0.1)
        loss.backward()
        optimizer.step()
    return loss.item()

# Train the model
for epoch in range(1, 11):
    loss = train()
    if epoch % 2 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')

In [ ]:
# Generate embeddings
model.eval()
with torch.no_grad():
    for graph in train_loader:
        _, _, mu, logvar = model(graph.x.double(), graph.edge_index)
        embeddings = model.reparameterize(mu, logvar)

In [ ]:
print(embeddings.shape)
embeddings = (embeddings - embeddings.min(0).values)/(embeddings.max(0).values - embeddings.min(0).values)

In [ ]:
df_map = pd.read_csv('./data/ReyZamuro_latlon.csv',index_col='field_numb')
df_map = df_map.drop('RZUA03b')

## Matriz de similitud

In [ ]:
embs = embeddings.numpy()
sim_mat = (embs@embs.T)
sim_mat

In [ ]:
#norms = np.linalg.norm(embs, axis=1)
#sim_mat = embs @ embs.T / (norms[:, np.newaxis] @ norms[np.newaxis, :])
#sim_mat

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from utils import edge_index_to_adjacency

In [ ]:
adj_graph = edge_index_to_adjacency(graph.edge_index)

In [ ]:
np.triu(sim_mat, k=-1)

In [ ]:
plt.figure(figsize=(12, 10))
plt.subplot(121)
sns.heatmap(np.triu(adj_graph, k=1), cmap="YlOrRd", square=True, cbar_kws=dict(use_gridspec=False,location="right",pad=0.01,shrink=0.5))
plt.title('Adjacency matrix of graph')
plt.xlabel("ARU Index")
plt.ylabel("ARU Index")
plt.subplot(122)
sns.heatmap(np.triu(sim_mat, k=1), cmap="YlOrRd", square=True, cbar_kws=dict(use_gridspec=False,location="right",pad=0.01,shrink=0.5))
plt.title('Simmilarity Matrix')
plt.xlabel("ARU Index")
plt.ylabel("ARU Index")

plt.show()

In [ ]:
plot_distance_matrix_heatmap(adj_graph)

In [ ]:
plot_distance_matrix_heatmap(sim_mat)

In [ ]:
# density metric:

print(np.mean(sim_mat)) #axis = 0: por nodos

In [ ]:
# connectivity metric (only if thresholded):

print(np.count_nonzero(sim_mat))

In [ ]:
# total weight (es density sin promediar)

print(np.sum(sim_mat))

In [ ]:
def floyd_warshall(matrix):
    """Implement Floyd-Warshall algorithm for all-pairs shortest paths"""
    n = len(matrix)
    dist = np.array(matrix, dtype=float)
    
    # Replace inf with large number for calculations
    dist[dist == float('inf')] = 1e9
    
    for k in range(n):
        for i in range(n):
            for j in range(n):
                dist[i][j] = min(dist[i][j], dist[i][k] + dist[k][j])
    
    return dist

In [ ]:
dist_matrix = floyd_warshall(sim_mat)
plot_distance_matrix_heatmap(dist_matrix)

In [ ]:
# diameter
print(np.max(dist_matrix))

In [ ]:
#average path
print(np.mean(dist_matrix))

In [ ]:
#Betweeness centrality

n = len(dist_matrix)
centrality = np.zeros(n)
for s in range(n):
    for t in range(n):
        if s != t:
            # Count shortest paths going through each vertex
            for v in range(n):
                if v != s and v != t:
                    if dist_matrix[s][t] == dist_matrix[s][v] + dist_matrix[v][t]:
                        centrality[v] += 1
value = np.mean(centrality)
print(value)

In [ ]:
dist_matrix